In [ ]:
import tensorflow as tf# Get the GPU device name.
device_name = tf.test.gpu_device_name()# The device name should look like the following:
if device_name == '/device:GPU:0':
 print('Found GPU at: {}'.format(device_name))
else:
 raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
#!cat /proc/cpuinfo

In [ ]:
import torch# If there's a GPU available...
if torch.cuda.is_available():
  device = torch.device("cuda") 
  print('There are %d GPU(s) available.' % torch.cuda.device_count()) 
  print('We will use the GPU:', torch.cuda.get_device_name(0))# If not...
else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
#!pip install transformers

In [ ]:
import re

In [ ]:
#deal with tensors
import torch   
import sys
#handling text data
from torchtext import data    
import pandas as pd
from torchtext.vocab import Vectors
#Reproducing same results
SEED = 2020

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True  

def remove_emojis(sentence):
  RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
  sentence = RE_EMOJI.sub(r'', sentence)
  return sentence

In [ ]:
train = pd.read_csv("/content/drive/My Drive/PFE/train.csv")
#train.drop(axis=1,columns=["Unnamed: 0","words"],inplace=True)
train["text"] = train["text"].apply(str)
#train["text"] = train["text"].apply(remove_emojis)
train.dropna(inplace=True)
train.head()

,text,label
0,thabal,1
1,hjhj m3alem y,1
2,يا سيدي علي عيشوشه واله تمثل مليح وتمثيلها مقنع,1
3,wsa5 tones,0
4,7lowa,1


In [ ]:
from transformers import BertTokenizer# Load the BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
train.sample(10)

,text,label
3741,ana bkit bravo walh,1
8757,h yhbel,1
6604,bravo rajel,1
6470,kamcha 5maj,0
6059,توفيق الحاس,0
8195,خيت ملا مناظر,0
12926,ماسط لاسط,0
2057,قناه الماخور التونسي اكبر فتنه في البلادو هي م...,0
579,mala 9o7b,0
3506,ه زعيم الكوميديا,1


In [ ]:
sentences = train.text.values
labels = train.label.values

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []# For every sentence...
for sent in sentences:
 # `encode` will:
 # (1) Tokenize the sentence.
 # (2) Prepend the `[CLS]` token to the start.
 # (3) Append the `[SEP]` token to the end.
 # (4) Map tokens to their IDs.
 encoded_sent = tokenizer.encode(
 sent, 
 # Sentence to encode.
 add_special_tokens = True, 
 # Add '[CLS]' and '[SEP]' # This function also supports truncation and conversion
 # to pytorch tensors, but we need to do padding, so we
 # can't use these features :( .
 #max_length = 128, # Truncate all sentences.
 #return_tensors = 'pt', # Return pytorch tensors.
 )

 # Add the encoded sentence to the list.
 input_ids.append(encoded_sent)# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (810 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (546 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (791 > 512). Running this sequence through the model will result in indexing errors


Original:  thabal
Token IDs: [101, 22794, 10264, 102]


In [ ]:
# We'll borrow the `pad_sequences` utility function to do this.
from keras.preprocessing.sequence import pad_sequences
# Set the maximum sequence length.
MAX_LEN = 128
print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))
# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post",
                          padding="post")
print('\Done.')


Padding/truncating all sentences to 128 values...

Padding token: "[PAD]", ID: 0
\Done.


In [ ]:
# Create attention masks
attention_masks = []
# For each sentence...
for sent in input_ids:
   # Create the attention mask.
   # - If a token ID is 0, then it's padding, set the mask to 0.
   # - If a token ID is > 0, then it's a real token, set the mask to 1.
   att_mask = [int(token_id > 0) for token_id in sent]
   # Store the attention mask for this sentence.
   attention_masks.append(att_mask)

In [ ]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
 random_state=2018, test_size=0.18)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,random_state=2, test_size=0.18)

In [ ]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)

train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# The DataLoader needs to know our batch size for training, so we specify it 

# For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32. 
batch_size = 16 
#Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
# Load BertForSequenceClassification,pretrained BERT model with a single linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
    # You can increase this for multi-class tasks. 
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    )
# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

N_trainable_params = count_parameters(model)



In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
 print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
 print('\n==== First Transformer ====\n')
for p in params[5:21]:
 print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
 print('\n==== Output Layer ====\n')
for p in params[-4:]:
 print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)

==== First Transformer ====

bert.embeddings.position_embeddings.weight                (512, 768)

==== First Transformer ====

bert.embeddings.token_type_embeddings.weight                (2, 768)

==== First Transformer ====

bert.embeddings.LayerNorm.weight                              (768,)

==== First Transformer ====

bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)

==== Output Layer ====

bert.encoder.layer.0.attention.self.query.bias                (768,)

==== Output Layer ====

bert.encoder.layer.0.attention.self.key.weight            (768, 768)

==== Output Layer ====

bert.encoder.layer.0.attention.self.key.bias                  (768,)

==== Output Layer ====

bert.encoder.layer.0.attention.self.value.weight  

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
 lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
 eps = 1e-8 # args.adam_epsilon - default is 1e-8.
 )
from transformers import get_linear_schedule_with_warmup
# Number of training epochs (authors recommend between 2 and 4)
epochs = 2
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
 num_warmup_steps = 0, 
 # Default value in run_glue.py
 num_training_steps = total_steps)

In [ ]:
import numpy as np
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
 pred_flat = np.argmax(preds, axis=1).flatten()
 labels_flat = labels.flatten()
 return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime
def format_time(elapsed):
 '''
 Takes a time in seconds and returns a string hh:mm:ss
 '''
 # Round to the nearest second.
 elapsed_rounded = int(round((elapsed)))

 # Format as hh:mm:ss
 return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
# Set the seed value all over the place to make this reproducible.
seed_val = 2
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# Store the average loss after each epoch so we can plot them.
loss_values = []# For each epoch...

t1 = time.time()

for epoch_i in range(0, epochs):

 # ========================================
 # Training
 # ========================================

 # Perform one full pass over the training set. print("")
 print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
 print('Training...') 
 # Measure how long the training epoch takes.
 t0 = time.time() 
 # Reset the total loss for this epoch.
 total_loss = 0 
 # Put the model into training mode. Don't be mislead--the call to 
 # `train` just changes the *mode*, it doesn't *perform* the training.
 # `dropout` and `batchnorm` layers behave differently during training
 # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
 model.train() 
 # For each batch of training data...
 for step, batch in enumerate(train_dataloader): 
   # Progress update every 40 batches.
  if step % 40 == 0 and not step == 0:
     # Calculate elapsed time in minutes.
     elapsed = format_time(time.time() - t0)
   # Report progress.
     print(' Batch {:>5,} of {:>5,}. Elapsed: {:}.'.format(step, len(train_dataloader), elapsed)) 
     # Unpack this training batch from our dataloader. 
 #
 # As we unpack the batch, we'll also copy each tensor to the GPU using the 
 # `to` method.
 #
 # `batch` contains three pytorch tensors:
 # [0]: input ids 
 # [1]: attention masks
 # [2]: labels 
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
   # Always clear any previously calculated gradients before performing a
 # backward pass. PyTorch doesn't do this automatically because 
 # accumulating the gradients is "convenient while training RNNs". 
 # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
  model.zero_grad()  
 # Perform a forward pass (evaluate the model on this training batch).
 # This will return the loss (rather than the model output) because we
 # have provided the `labels`.
 # The documentation for this `model` function is here: 
 # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
  outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, 
                  labels=b_labels)

 # The call to `model` always returns a tuple, so we need to pull the 
 # loss value out of the tuple.
  loss = outputs[0] 
 # Accumulate the training loss over all of the batches so that we can
 # calculate the average loss at the end. `loss` is a Tensor containing a
 # single value; the `.item()` function just returns the Python value 
 # from the tensor.
  total_loss += loss.item() 
  # Perform a backward pass to calculate the gradients.
  loss.backward() 
  # Clip the norm of the gradients to 1.0.
 # This is to help prevent the "exploding gradients" problem.
  torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) 
 # Update parameters and take a step using the computed gradient.
 # The optimizer dictates the "update rule"--how the parameters are
 # modified based on their gradients, the learning rate, etc.
  optimizer.step() 
  # Update the learning rate.
  scheduler.step() 
  # Calculate the average loss over the training data.
 avg_train_loss = total_loss / len(train_dataloader) 

 # Store the loss value for plotting the learning curve.
 loss_values.append(avg_train_loss) 
 print("")
 print(" Average training loss: {0:.2f}".format(avg_train_loss))
 print(" Training epcoh took: {:}".format(format_time(time.time() - t0)))
 
 # ========================================
 # Validation
 # ========================================
 # After the completion of each training epoch, measure our performance on
 # our validation set. print("")
 print("Running Validation...") 
 t0 = time.time() 
 # Put the model in evaluation mode--the dropout layers behave differently
 # during evaluation.
 model.eval() 
 # Tracking variables 
 eval_loss, eval_accuracy = 0, 0
 nb_eval_steps, nb_eval_examples = 0, 0 
 # Evaluate data for one epoch
 for batch in validation_dataloader:

 # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)

 # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch

 # Telling the model not to compute or store gradients, saving memory and
 # speeding up validation
  with torch.no_grad():  
    # Forward pass, calculate logit predictions.
 # This will return the logits rather than the loss because we have
 # not provided labels.
 # token_type_ids is the same as the "segment ids", which 
 # differentiates sentence 1 and 2 in 2-sentence tasks.
 # The documentation for this `model` function is here: 
 # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

 # Get the "logits" output by the model. The "logits" are the output
 # values prior to applying an activation function like the softmax.
    logits = outputs[0] 
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

  # Calculate the accuracy for this batch of test sentences.
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

 # Accumulate the total accuracy.
    eval_accuracy += tmp_eval_accuracy 
    # Track the number of batches
    nb_eval_steps += 1 
    # Report the final accuracy for this validation run.
 print(" Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
 print(" Validation took: {:}".format(format_time(time.time() - t0)))
 print("")



======== Epoch 1 / 2 ========
Training...
 Batch    40 of   701. Elapsed: 0:00:08.
 Batch    80 of   701. Elapsed: 0:00:17.
 Batch   120 of   701. Elapsed: 0:00:25.
 Batch   160 of   701. Elapsed: 0:00:33.
 Batch   200 of   701. Elapsed: 0:00:42.
 Batch   240 of   701. Elapsed: 0:00:50.
 Batch   280 of   701. Elapsed: 0:00:59.
 Batch   320 of   701. Elapsed: 0:01:07.
 Batch   360 of   701. Elapsed: 0:01:15.
 Batch   400 of   701. Elapsed: 0:01:24.
 Batch   440 of   701. Elapsed: 0:01:32.
 Batch   480 of   701. Elapsed: 0:01:40.
 Batch   520 of   701. Elapsed: 0:01:49.
 Batch   560 of   701. Elapsed: 0:01:57.
 Batch   600 of   701. Elapsed: 0:02:05.
 Batch   640 of   701. Elapsed: 0:02:14.
 Batch   680 of   701. Elapsed: 0:02:22.

 Average training loss: 0.54
 Training epcoh took: 0:02:26
Running Validation...
 Accuracy: 0.80
 Validation took: 0:00:09

======== Epoch 2 / 2 ========
Training...
 Batch    40 of   701. Elapsed: 0:00:08.
 Batch    80 of   701. Elapsed: 0:00:17.
 Batch   120

In [ ]:
elapsed = format_time(time.time() - t1)
print("Training complete!")

Training complete!


In [ ]:
import plotly.express as px
f = pd.DataFrame(loss_values)
f.columns=['Loss']
fig = px.line(f, x=f.index, y=f.Loss)
fig.update_layout(title='Training loss of the Model',xaxis_title='Epoch',yaxis_title='Loss')
fig.show()

In [ ]:
##TESTING

test = pd.read_csv("/content/drive/My Drive/PFE/test.csv")

In [ ]:
##TESTING

test["text"] = test["text"].apply(str)
#test["text"] = test["text"].apply(remove_emojis)
test.head()

,text,label
0,هو المنظر هاذا يستاهل كل شي نهار كامل وهو يعوع...,0
1,marzen damk pf 3,0
2,اله لتربحهم,0
3,hahaha bravo walah ma9tla tha7k d,1
4,حاحه حلوي عالاقل واحد منا يلقي علاس يتفرح ه,1


In [ ]:
sentences = test.text.values
labels = test.label.values

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
# For every sentence...
for sent in sentences:
 # `encode` will:
 # (1) Tokenize the sentence.
 # (2) Prepend the `[CLS]` token to the start.
 # (3) Append the `[SEP]` token to the end.
 # (4) Map tokens to their IDs.
 encoded_sent = tokenizer.encode(
 sent,
 add_special_tokens = True, 
 )

 # Add the encoded sentence to the list.
 input_ids.append(encoded_sent)# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (766 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (797 > 512). Running this sequence through the model will result in indexing errors


Original:  هو المنظر هاذا يستاهل كل شي نهار كامل وهو يعوعش و عامل فاها صوت وهو واحد ماسط  مسخ  حثاله سوق و ماهوش متربي والمشكله في اعلام العار و البرامج السوقيه التافهه لي تجيب فيه و في امثاله في التلفزه
Token IDs: [101, 1297, 29836, 1270, 23673, 22192, 15915, 29829, 17149, 1297, 25573, 29822, 25573, 1300, 29824, 29817, 25573, 14157, 23673, 1293, 23673, 1283, 14498, 1296, 14157, 25573, 17149, 1293, 25573, 22192, 23673, 1298, 14157, 29836, 1300, 29830, 29836, 29830, 29825, 1298, 1288, 25573, 22192, 23673, 1291, 25573, 14157, 25573, 1284, 29836, 29817, 1298, 14157, 29836, 1298, 25573, 29820, 15394, 1295, 25573, 29824, 29828, 1295, 29824, 29821, 1276, 29818, 25573, 23673, 14157, 1282, 29836, 29834, 1298, 1295, 25573, 14157, 29836, 29825, 1295, 29817, 17149, 29816, 14498, 1298, 25573, 23673, 22192, 29825, 29835, 23673, 14157, 1291, 14498, 1270, 29830, 23673, 25573, 22192, 1270, 23673, 29830, 25573, 17149, 1298, 1270, 23673, 29816, 17149, 25573, 22192, 29819, 1270, 23673, 29824, 29836, 2983

In [ ]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post",padding="post")

In [ ]:
test_inputs = torch.tensor(input_ids)#.to(device)
test_labels = torch.tensor(labels)#.to(device)

#test_labels.shape

In [ ]:
# Create the DataLoader for our test set.
test_data = TensorDataset(test_inputs, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
model.eval()
i = 0
for batch in test_dataloader:
  i += 1

 # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)

 # Unpack the inputs from our dataloader
  b_input_ids, b_labels = batch

 # Telling the model not to compute or store gradients, saving memory and
 # speeding up validation
  with torch.no_grad():  
    outputs = model(b_input_ids, token_type_ids=None)
    #print(outputs)
    logits = outputs[0] 
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    if i>1:
      #print("logits:",logits.shape,"**********predictions:",predictions.shape)
      predictions = np.append(predictions,logits,axis=0)
      labels = np.append(labels,label_ids)
    else:
      predictions = logits
      labels = label_ids


accuracy = flat_accuracy(predictions, labels)
print(accuracy)

0.8874484384207425
